<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">AG Dynamics of the Earth</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Jupyter notebooks</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
</tr>
</table>

# Dynamic systems: 9. Viscous material
## Hagen-Poiseuille with `icoScalarTransportFoam`
----
*Georg Kaufmann,
Geophysics Section,
Institute of Geological Sciences,
Freie Universität Berlin,
Germany*

**In this notebook, we will learn to**

- create and compile our own `openFOAM` program.
- establish a typical **Hagen-Poiseuille flow** profile in a rectangular channel
in `openFoam`, using our new `icoScalarTransportFoam` tool. 

**Prerequisites:** (text)

**Result:** You should get a figure similar to
<img src="images/HagenPoiseuille_run3D_T1.jpg" style=width:20cm>

<a href="#top">**Table of contents**</a>

1. [Solver and equations](#one)
2. [Coding](#two)
3. [Implementation](#three)
4. [Running](#four)
5. [Post-processing](#five)


<div id="one"></div>

----
## 1. Solver and equations

We will merge the tasks of `icoFoam` and `scalarTransportFoam` to calculate both
flow and transport of a passive scalar in a model. The new `icoScalarTransportFoam` will be

- transient
- incompressible
- laminar

and solves the continuity and momentum equations:
$$
\begin{array}{rcl}
\nabla \cdot \vec{u} &=& 0 \\
\frac{\partial \vec{u}}{\partial t}
+ \left( \vec{u} \cdot \nabla \right) \vec{u}
- \nabla \cdot \frac{\eta}{\rho} \nabla \vec{u}
&=& - \nabla \frac{p}{\rho} \\
\frac{\partial T}{\partial t} 
+ \vec{u} \cdot \nabla T  
- \nabla \cdot \kappa_h \nabla T &=& 0
\end{array}
$$
with
$\rho$ [kg/m$^3$] density,
$\vec{u}$ [m/s] velocity,
$t$ [s] time,
$p$ [Pa] pressure,
$T$ [K] temperature,
$\kappa$ [m$^2$/s] thermal diffusivity, 
$\eta$ [Pas] dynamic viscosity, and with
$\nu={{\eta}\over{\rho}}$ [m$^2$/s] kinematic viscosity.

The equations are then solved by `icoScalarTransportFoam`, which we have to create first.

<div id="two"></div>

----
## 2. Coding

We start coding a new tool with first understanding, how `openFOAM` is organised.


### 2.1 Simple re-compilation of  `icoFoam`


We check two shell environment variables:

~~~bash
$ echo $WM_PROJECT_DIR
/opt/openfoam9

$ echo $FOAM_APPBIN
/opt/openfoam9/platforms/linux64GccDPInt32Opt/bin
~~~

The first variables indicates where the entire `openFOAM` package resides, the second
one points to the binaries. Try ...

~~~bash
$ ls $FOAM_APPBIN
~~~

The source codes for the `openFOAM` programs reside under

~~~
|-- $WM_PROJECT_DIR/applications
  |-- solvers
      |-- basic
          |-- laplacianFoam
          |-- potentialFoam
          |-- scalarTransportFoam
      |-- incompressible
          |-- icoFoam
          |-- simpleFoam
          |-- shallowWaterFoam
          |-- ...
      |-- ...
  |-- ...
~~~

We identify the `icoFoam`and `scalarTransportFoam` directories. The structure in the
directories is always similar, e.g. for `icoFoam`:

~~~
|-- icoFoam
  |-- createFields.H
  |-- icoFoam.C
  |-- Make
      |-- files
      |-- options
~~~

To test compiling new code, we make a copy of the `icoFoam` source code to our local
work folder, rename  the directory to `myIcoFoam` and the C-file to `myIcoFoam.C`.

We then change the `files` file from

~~~
icoFoam.C
  
EXE = $(FOAM_APPBIN)/icoFoam
~~~

to

~~~
myIcoFoam.C
  
EXE = /home/mybin/myIcoFoam
~~~

Note that the folder `/home/mybin/bin` has to exist!

In the `myIcoFoam` directory, we then type:

~~~bash
$ wclean
$ wmake
~~~

The first command cleans the directory, the second one complies the renamed code and
moves it to the defined bin-directory.

**Done!** We can test the newly compiled code `myIcoFoam` with one of our old
Hagen-Poiseuille runs ...

### 2.2 Creation and compilation of  `icoScalarTransportFoam`

This the re-compilation of our `myIcoFoam` worked, we make another copy of the `icoFoam` directory
and rename it to `icoScalarTransportFoam` (and all necessary changes in the files ...).

Then, we need to modity both files
- createFields.H
- icoScalarTransportFoam.C

to include the scalar transport equation ...

- `icoScalarTransportFoam.C`

We first add the scalar transport equation for the temperature to the new `icoScalarTransportFoam.C` code.
We define a new scalar matrix equation **TEqn**, then simply rebuild the transport equation with
the `openFOAM` makros. Finally, relaxation of the equation is applied, and then the equation is
solved:

~~~
// passive scalar transport of T
    fvScalarMatrix TEqn
    (
        fvm::ddt(T)
        + fvm::div(phi, T)
        - fvm::laplacian(DT, T)
    );

    TEqn.relax();
    TEqn.solve();
~~~



<details><summary><b>> Show code</b></summary>
int main(int argc, char *argv[])
{
    argList::addNote
    (   
        "Transient solver for incompressible, laminar flow"
        " of Newtonian fluids, additing passive scalar transport."
    );

    #include "postProcess.H"

    #include "addCheckCaseOptions.H"
    #include "setRootCaseLists.H"
    #include "createTime.H"
    #include "createMesh.H"

    pisoControl piso(mesh);

    #include "createFields.H"
    #include "initContinuityErrs.H"
    // * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //
    Info<< "\nStarting time loop\n" << endl;

    while (runTime.loop())
    {
        Info<< "Time = " << runTime.timeName() << nl << endl;
        #include "CourantNo.H"
        // Momentum predictor
        fvVectorMatrix UEqn
        (
            fvm::ddt(U)
          + fvm::div(phi, U)
          - fvm::laplacian(nu, U)
        );
        if (piso.momentumPredictor())
        {
            solve(UEqn == -fvc::grad(p));
        }
        // --- PISO loop
        while (piso.correct())
        {
            volScalarField rAU(1.0/UEqn.A());
            volVectorField HbyA(constrainHbyA(rAU*UEqn.H(), U, p));
            surfaceScalarField phiHbyA
            (
                "phiHbyA",
                fvc::flux(HbyA)
              + fvc::interpolate(rAU)*fvc::ddtCorr(U, phi)
            );
            adjustPhi(phiHbyA, U, p);
            // Update the pressure BCs to ensure flux consistency
            constrainPressure(p, U, phiHbyA, rAU);

            // Non-orthogonal pressure corrector loop
            while (piso.correctNonOrthogonal())
            {
                // Pressure corrector

                fvScalarMatrix pEqn
                (
                    fvm::laplacian(rAU, p) == fvc::div(phiHbyA)
                );

                pEqn.setReference(pRefCell, pRefValue);

                pEqn.solve(mesh.solver(p.select(piso.finalInnerIter())));

                if (piso.finalNonOrthogonalIter())
                {
                    phi = phiHbyA - pEqn.flux();
                }
            }
            #include "continuityErrs.H"
            U = HbyA - rAU*fvc::grad(p);
            U.correctBoundaryConditions();
        }
        
        // passive scalar transport of T
        fvScalarMatrix TEqn
        (
            fvm::ddt(T)
            + fvm::div(phi, T)
            - fvm::laplacian(DT, T)
        );

        TEqn.relax();
        TEqn.solve();


        runTime.write();

        runTime.printExecutionTime(Info);
    }

    Info<< "End\n" << endl;

    return 0;
}    
</details>

- `createFields.H`

We then add temperature $T$ [K] and the thermal diffusivity $\kappa_T$ [m$^2$/s] to the 
new `createFields.H` file: 

- We first read the thermal diffusivity as $DT$ from tthr `transportProperties` dictionary in the 
`constants` folder:

~~~
dimensionedScalar DT
(
    "DT",
    dimViscosity,
    transportProperties
);
~~~

- We then read the temperatures as scalar field $T$ from the different time directories:

~~~
Info<< "Reading field T\n" << endl;
volScalarField T
(
    IOobject
    (
        "T",
        runTime.timeName(),
        mesh,
        IOobject::MUST_READ,
        IOobject::AUTO_WRITE
    ),
    mesh
);
~~~

<details><summary><b>> Show code</b></summary>
Info<< "Reading transportProperties\n" << endl;
  
IOdictionary transportProperties
(
    IOobject
    (
        "transportProperties",
        runTime.constant(),
        mesh,
        IOobject::MUST_READ_IF_MODIFIED,
        IOobject::NO_WRITE
    )
);

dimensionedScalar nu
(
    "nu",
    dimViscosity,
    transportProperties
);

dimensionedScalar DT
(
    "DT",
    dimViscosity,
    transportProperties
);

Info<< "Reading field p\n" << endl;
volScalarField p
(
    IOobject
    (
        "p",
        runTime.timeName(),
        mesh,
        IOobject::MUST_READ,
        IOobject::AUTO_WRITE
    ),
    mesh
);

Info<< "Reading field U\n" << endl;
volVectorField U
(
    IOobject
    (
        "U",
        runTime.timeName(),
        mesh,
        IOobject::MUST_READ,
        IOobject::AUTO_WRITE
    ),
    mesh
);

Info<< "Reading field T\n" << endl;
volScalarField T
(
    IOobject
    (
        "T",
        runTime.timeName(),
        mesh,
        IOobject::MUST_READ,
        IOobject::AUTO_WRITE
    ),
    mesh
);


#include "createPhi.H"


label pRefCell = 0;
scalar pRefValue = 0.0;
setRefCell(p, mesh.solutionDict().subDict("PISO"), pRefCell, pRefValue);
mesh.setFluxRequired(p.name());
</details>

After checking changes in the `Make`sub-directory, we clean and compile the new code...

~~~bash
$ wclean
$ wmake
~~~~

... then test it in `run3D_T1`.

<div id="three"></div>

----
## 3.  Implementation

We consider flow along a channel 10 m long, 1 m in cross section:
<img src="images/HagenPoiseuille.jpg" style=width:15cm>
Black numbers mark sizes, red numbers are vertex numbers.

For the calculation of velocities and pressure, we have an inflow face (0 4 7 3) with fixed velocity
in $y$ direction, and an outflow face (1 2 6 2) with fixed (kinematic) pressure.
The numbering of faces mark the **unit normal vector** in a right-hand side sense, thus for both
faces, the unit vector points outwards.

The other faces (left,right,top,bottom) will be defined as **no-slip** boundaries, 
thus they will be defined with velocity zero.

We have to set up the **mesh**, the **initial conditions**, a **temperature anomaly**, then **run** the program.
This is done for the 3D case in the following.

### Directory structure and files

~~~
HagenPoiseuille_icoScalarTransportFoam
|-- 0
  |-- U
  |-- p
  |-- T
|- constant
  |-- transportProperties
|- system
  |-- blockMesh
  |-- controlDict
  |-- fvSchemes
  |-- fvSolution
~~~

----

- `system/blockMeshDict`
    - We use no scaling (`convertToMeters`)
    - Eight vertices are defined (`vertices`), use the figure above. The model domain is 10m long in $y$ direction, 
    and has dimensions of $\pm$0.5m in $x$ and $z$ direction.
    - From the eight vertices, a single block (`blocks`) is created as mesh,
    discretised with 100 points in $y$ direcion, ans 10 points in $x$ and $z$ directions.
    Choose direction of points such that the normal vectors are pointing outside! Create one block, withh 1000 elements in $x$ direction.
    - All edges are straight, thus the keyword `edges` is empty.
    - The boundries are defined (`boundary`). Only two faces, inflow and outflow side, are relevant, 
    and defined as patches. The other four faces (left,right,top,bottom), are no-slip boundaries.

<details><summary><b>> Show code</b></summary>

~~~
convertToMeters 1;

vertices
(
    (-0.5   0.0  -0.5)
    ( 0.5   0.0  -0.5)
    ( 0.5  10.0  -0.5)
    (-0.5  10.0  -0.5)
    (-0.5   0.0   0.5)
    ( 0.5   0.0   0.5)
    ( 0.5  10.0   0.5)
    (-0.5  10.0   0.5)
);

blocks
(
    hex (0 1 2 3 4 5 6 7) ( 10 100 10) simpleGrading (1 1 1)
);

edges
(
);

boundary
(
    inlet
    {
        type patch;
        faces
        (
            (0 1 5 4)
        );
    }
    outlet
    {
        type patch;
        faces
        (
            (2 3 7 6)
        );
    }
    left
    {
        type wall;
        faces
        (
            (0 4 7 3)
        );
    }
    right
    {
        type wall;
        faces
        (
            (1 2 6 5)
        );
    }
    bottom
    {
        type wall;
        faces
        (
            (0 3 2 1)
        );
    }
    top
    {
        type wall;
        faces
        (
            (4 5 6 7)
        );
    }
);

mergePatchPairs
(
);
~~~
</details>

----

- `0/U`
    - Dimensions are set for velocities (`dimensions`).
    - Set all velocities to zero, except the inlet face.
    - At the `inlet`, set velocity to 1 m/s in $y$ direction.
    - At the `outlet`, set velocity to zero gradient.
    - Fix velocities along other edges to a fixed value and to zero.

<details><summary><b>> Show code</b></summary>

~~~
dimensions      [0 1 -1 0 0 0 0];
internalField   uniform (0 0 0);
boundaryField
{
    inlet
    { type            fixedValue;
      value           uniform (0 1 0);
    }

    outlet
    { type      zeroGradient;
    }

    left
    { type            fixedValue;
      value           uniform (0 0 0);
    }

    right
    { type            fixedValue;
      value           uniform (0 0 0);
    }

    bottom
    { type            fixedValue;
      value           uniform (0 0 0);
    }

    top
    { type            fixedValue;
      value           uniform (0 0 0);
    }   
}
~~~
</details>

----

- `0/p`
    - Dimensions are set for pressure (`dimensions`).
    - Set all pressures to zero gradient, 
    - except the outlet face, where a fixed value is used
    and set to 0 m/s.
    - Note that for `icoFoam`, which is an incompressible solver, the pressure is scaled by density:
    $$
    p^* = \frac{p}{\rho}
    $$
    with $p$ [Pa] pressure and $\rho$ [kg/m$^3$] density, thus $p^*$ [m$^2$/s$^2$].

<details><summary><b>> Show code</b></summary>

~~~
dimensions      [0 2 -2 0 0 0 0];

internalField   uniform 0;

boundaryField
{
    inlet
    { type            zeroGradient;
    }

    outlet
    { type            fixedValue;
      value           $internalField;
      //type        zeroGradient;
    }

    left
    { type      zeroGradient;           
    }

    right
    { type      zeroGradient;           
    }

    bottom
    { type      zeroGradient;
    }

    top
    { type      zeroGradient;
    }
}
~~~
</details>

----

- `0/T`
    - Dimensions are set for temperature (`dimensions`).
    - Set all temperatures along outside faces to zero gradient, 
    - set interior temperature field to 10K.

<details><summary><b>> Show code</b></summary>
dimensions      [0 0 0 1 0 0 0];

internalField   uniform 10;

boundaryField
{
    inlet
    { type            zeroGradient; }

    outlet
    { type            zeroGradient; }

    left
    { type            zeroGradient; }

    right
    { type            zeroGradient; }

    bottom
    { type            zeroGradient; }

    top
    { type            zeroGradient; }
}
</details>


----

- `constant/transportProperties`
    - define **kinematic viscosity** $\nu=\frac{\eta}{\rho}$ [m$^2$/s] from
    - **dynamic viscosity** $\eta$ [Pa] and
    - **density** $\rho$ [kg/m$^3$]
    - The value is choosen to remain in the **laminar** flow regime, which is
    defined via the **Reynolds number** as
    $$
    Re = \frac{\rho u L}{\eta} = \frac{u L}{\nu}
    $$
    For $Re=100$, $u=1$m/s, and $L\simeq 1$m, we therefore need $\nu=0.01$m$^2$/s
    (which is way to large for water!). 

<details><summary><b>> Show code</b></summary>

~~~
nu              nu [ 0 2 -1 0 0 0 0 ] 0.01; // kinematic viscosity
DT              DT [ 0 2 -1 0 0 0 0 ] 1e-2; // thermal diffusivity
~~~
</details>

----

- `system/setFieldsDict`
    - Used to add a blob of 20K along the left inflow boundary.

<details><summary><b>> Show code</b></summary>
defaultFieldValues ( volScalarFieldValue T 10. );

regions         ( boxToCell { box (-0.4 0 -0.4) (0.4 1 0.4) ; fieldValues ( volScalarFieldValue T 20. ) ; } );
</details>

----

- `system/controlDict`

    - Our example starts from (`startFrom`), set to time 0 (`startTime`).
    - We run the example for 20 seconds (`stopAt`), set with (`endTime`).
    - Time step is set to 0.05 seconds (`deltaT`).
    - We write the solution every second (`writeInterval`) of
    simulation time (`writeControl runTime`).
    - We keep all solution directories (`purgeWrite 0`).
    - We save the solution in ascii format (`writeFormat ascii`).
    - Write precision is set to 8 digits (`writePrecision 8`). 
    - The option `runTimeModifiable` is on, we therefore can modify all
    these entries while we are running the simulation.

<details><summary><b>> Show code</b></summary>

~~~
application     icoFoam;
startFrom       startTime;
startTime       0;
stopAt          endTime;
endTime         20;
deltaT          0.05;
writeControl    runTime;
writeInterval   1;
purgeWrite      0;
writeFormat     ascii;
writePrecision  8;
writeCompression off;
timeFormat      general;
timePrecision   6;
runTimeModifiable true;
~~~
</details>

----

- `system/fvSchemes`

    - Time is discretised with the Euler method (`ddtSchemes`).
    - Gradients are solved with the Gauss linear methods (`gradSchemes`).
    - Divergence is solved with a linear interpolation method (`divSchemes`).
    - Laplacians are solved with a Gauss linear method (`laplacianSchemes`).

<details><summary><b>> Show code</b></summary>

~~~
ddtSchemes
{
    default         Euler;
}

gradSchemes
{
    default         Gauss linear;
    //default         cellMDLimited Gauss linear 1;

    grad(p)         Gauss linear;
}

divSchemes
{
    default         none;
    div(phi,U)      Gauss linear;
    //div(phi,U)      Gauss linearUpwind default;
    //div(phi,U)      Gauss vanLeer;
    //div(phi,U)      Gauss upwind;
}

laplacianSchemes
{
    default         Gauss linear orthogonal;
    //default         Gauss linear limited 0.5;
    //default         Gauss linear limited 0;
}

interpolationSchemes
{
    default         linear;
}

snGradSchemes
{
    default         orthogonal;
    //default         limited 0.5;
    //default         limited 0;
}
~~~
</details>

----

- `system/fvSolution`

    - Pressure $p$ is solved with the **GAMG** method to a given absolute tolerance,
    and a given relative tolerance.
    - For the final pressure calculation in the **PISO** loop, a more accurate
    tolerance is required.
    - Velocities $U$ are solved with the **PBiCG** method and **DILU** preconditioning,
    and toleances are set.
    - The **PISO** loop is run only once.

<details><summary><b>> Show code</b></summary>

~~~
solvers
{
    p
    {
        solver           GAMG;
        tolerance        1e-6;
        relTol           0.01;
        smoother         GaussSeidel;
        nPreSweeps       0;
        nPostSweeps      2;
        cacheAgglomeration on;
        agglomerator     faceAreaPair;
        nCellsInCoarsestLevel 100;
        mergeLevels      1;
    }

    pFinal
    {
        $p;
        relTol          0;
    }

    U
    {
        solver          PBiCG;
        preconditioner  DILU;
        tolerance       1e-08;
        relTol          0;
    }
}

PISO
{
    nCorrectors     1;
    nNonOrthogonalCorrectors 1;
    //pRefCell        0;
    //pRefValue       0;
}
~~~
</details>

<div id="four"></div>

----
## 4. Running

Running a particular example is done with the following set of commands:

~~~
$ foamCleanTutorials
$ blockMesh
$ setFields
$ icoScalarTransportFoam
~~~

If the run was successful, create a dummy file `show.foam` in the directory and
open the run in `paraview`.

<img src="images/HagenPoiseuille_run3D_T1.jpg" style=width:20cm>

<div id="five"></div>

----
## 5. Post-processing: Profiles

as before ...

----